In [21]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)



from Camera import Camera
from Frame import Frame
import Plotters
import Utils as Utils

from render.GaussianSplat import GaussianSplat


import numpy as np
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import Plotters
import plotly
import plotly.graph_objects as go
import scipy.io
import pandas as pd
from Render import Render
from plyfile import PlyData

import pickle
import numpy as np
import math

import matplotlib.pyplot as plt
import Utils
%matplotlib qt


path = 'C:/Users/Roni/Documents/gs_input/frames_model.pkl'


# dict_path = 'D:/Documents/data_for_gs/fly_gray/dict/frames_model.pkl'







path_output = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/hull_output'
path_output = 'D:/Documents/gaussian_model_output/'

dict_path  = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov7_2024_11_12_darkan/frames_model.pkl'
image_path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov7_2024_11_12_darkan/'
model_name = 'fly_to_bee'
file_name = 'bee_model_dense_10000'


# dict_path  = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov30_2024_11_12_darkan/frames_model.pkl'
# image_path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov30_2024_11_12_darkan/'
# model_name = 'only_fly'
# file_name = 'fly_model'

model_name = 'fly_to_fly'
file_name = 'fly_model'

# dict_path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov7_2024_11_12_darkan/frames_model.pkl'
# image_path =  'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov7_2024_11_12_darkan/'
# model_name = '3dgs_bee'
# file_name = 'bee_try'
path_output = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/'
path_output = 'D:/Documents/gaussian_model_output/'

dict_path  = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov7_2024_11_12_darkan/frames_model.pkl'
image_path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov7_2024_11_12_darkan/'
model_name = 'fly_to_bee'
file_name = 'bee_model'

# model_name = 'fly_fast'
# file_name = 'fly_model_fast'

# dict_path  = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov30_2024_11_12_darkan/frames_model.pkl'
# image_path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov30_2024_11_12_darkan/'

# model_name = 'fly_evaluation'
# file_name = 'fly_model_fast'
# dict_path  = 'D:/Documents/data_for_gs/mov1_2023_08_09_60ms/dict/frames_model.pkl'
# image_path = 'D:/Documents/data_for_gs/mov1_2023_08_09_60ms/'


path_angles = f'{path_output}/{model_name}/{file_name}_angles.pkl'
path_results = f'{path_output}/{model_name}/{file_name}_angles.pkl'


# download model_run localy
path = f'D:/Documents/gaussian_model_output/{model_name}/{file_name}.pkl'
if os.path.exists(f'{path}'):
    with open(path, 'rb') as handle:
        output_angles_weights = pickle.load(handle)

iteration = 2200

frame0 = 1015
frame_end = 1029
weight_flag = False

with open(dict_path,'rb') as f:
    frames = pickle.load(f)

vertices_list = []
image_list = []
weights_list = []
gaussian_list = []
idx_parts = []
xyz_rotated = []
for frame in range(frame0,frame_end):
    ew_to_lab = frames[frame][1][list(frames[frame][1].keys())[0]]['ew_to_lab']
    input_dir = f'{path_output}/{model_name}/{file_name}'
    input_file = f'{path_output}/{model_name}/{frame}/{file_name}/point_cloud/iteration_{iteration}/point_cloud.ply'
    vertices = PlyData.read(input_file)["vertex"]
    xyz = np.column_stack((vertices['x'],vertices['y'],vertices['z']))
    vertices_list.append(xyz)
    xyz_rotated.append((ew_to_lab @ xyz.T).T)

    frames_per_cam = [Frame(image_path,frame,cam, frames_dict = frames)  for cam in range(4)]
    image_list.append(frames_per_cam)
    if os.path.exists(f'{input_dir}_results.pkl'):
        with open(f'{input_dir}_results.pkl', 'rb') as handle:
            output_angles_weights = pickle.load(handle)
        weights_list.append(output_angles_weights['weights'])
        weight_flag = True
        idx_parts.append([np.sum(output_angles_weights['weights'][frame - frame0][iteration][:,idx:idx + 3],axis = 1) == 1 for idx in range(0,9,3)])
        color_list = ['lime','crimson','dodgerblue']
        color_list_2d = ['lime','crimson','dodgerblue']





# frames_per_cam = [Frame(image_path,frame,cam_num, frames_dict = frames) for cam_num in range(4)]


In [11]:
frame = 370

# input_file = f'D:/Documents/gaussian_model_output/mdoel_le_deform/{frame}/{file_name}/point_cloud/iteration_900/point_cloud.ply'


frame_data = image_list[frame - frame0] 

ew_to_lab = frames[frame][1][list(frames[frame][1].keys())[0]]['ew_to_lab']
xyz = vertices_list[frame - frame0]
# xyz_rotated = (ew_to_lab @ xyz.T).T

gs = GaussianSplat(input_file,image_size = frame_data[0].image_size)
grayscale = (gs.color[:,0] - gs.color[:,0].min()) / (gs.color[:,0].max() - gs.color[:,0].min())
grayscale = grayscale[grayscale <1]


if weight_flag == False:
    idx_parts = list(range(len(xyz)))
    color_list = [grayscale]
    color_list_2d = ['red']

fig = go.Figure()
Plotters.scatter3d(fig,xyz_rotated[frame - frame0],grayscale,3,'body',show_colorbar = False)

for idx,color in zip(range(len(idx_parts[frame - frame0])),color_list):   
    Plotters.scatter3d(fig,xyz_rotated[frame - frame0][idx_parts[frame - frame0][idx],:],color,3,'body',show_colorbar = False)

fig.show()
fig.write_html(f'{path_output}/plot_hull.html')

ax = None
for idx,color in zip(range(len(idx_parts[frame - frame0])),color_list_2d): 
    ax = Plotters.plot_projections(xyz[idx_parts[frame - frame0][idx],:],frame_data,color = color,ax = ax)





d:\Documents\model_gaussian_splatting\Plotters.py:71: UserWarning:

No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored



In [11]:
path_output

'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/hull_output'

In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs = axs.flatten()
ims = []

h, w = image_list[0][0].im.size

# Set axis limits and titles once
for cam in range(4):
    axs[cam].set_xlim(0, w)
    axs[cam].set_ylim(h, 0)
    axs[cam].set_title(f'Camera {cam}')

for i in range(frame_end - frame0):  # loop over frames
    frame_artists = []
    for cam in range(4):
        ax = axs[cam]

        xy = image_list[i][cam].project_with_proj_mat(vertices_list[i])
        im = ax.imshow(image_list[i][cam].im, cmap='gray', animated=True)

        scatter = [ax.scatter(xy[idx_parts, 0], xy[idx_parts, 1], c=color, s=2, animated=True) for idx_parts,color in zip(idx_parts[i],color_list)]
        frame_artists.extend([im, scatter[0], scatter[1], scatter[2]])

    ims.append(frame_artists)

ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True, repeat_delay=10)
plt.tight_layout()
plt.show()


In [20]:
import plotly.graph_objects as go
import numpy as np

# === CONFIG ===
frames = range(frame0, frame_end)
output_path = f'{path_output}/{model_name}/animated_plot_bee.html'


# === HELPERS ===

def create_scatter3d(xyz, idx_parts, color):
    """Create a single 3D scatter trace for a specific part."""
    return go.Scatter3d(
        x=xyz[idx_parts, 0],
        y=xyz[idx_parts, 1],
        z=xyz[idx_parts, 2],
        mode="markers",
        marker=dict(size=2, opacity=1, color=color, colorscale='gray'),
    )


def get_global_bounds(xyz_list):
    """Compute global min and max coordinates over all frames for consistent axis scaling."""
    stacked = np.vstack(xyz_list)
    return np.min(stacked, axis=0), np.max(stacked, axis=0)


def create_frame(xyz_frame, idx_parts_list, color_list, frame_name):
    """Create one animation frame with all parts for a given timestep."""
    data = [
        create_scatter3d(xyz_frame, idx_parts, color)
        for idx_parts, color in zip(idx_parts_list, color_list)
    ]
    return go.Frame(data=data, name=frame_name)


def create_play_pause_buttons():
    """Return Play/Pause button definitions for animation."""
    return [
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}],
                    "label": "Play",
                    "method": "animate",
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                    "label": "Pause",
                    "method": "animate",
                },
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top",
        }
    ]


def create_slider(frames_range):
    """Create a slider object to control the animation."""
    return [
        {
            "active": 0,
            "steps": [
                {
                    "args": [[str(i)], {"frame": {"duration": 100, "redraw": True}, "mode": "immediate"}],
                    "label": str(frame),
                    "method": "animate",
                }
                for i, frame in enumerate(frames_range)
            ],
        }
    ]


# === MAIN FUNCTION ===

def create_3d_animation(xyz_all_frames, idx_parts_list, color_list):
    """Build and show the 3D animation."""
    min_xyz, max_xyz = get_global_bounds(xyz_all_frames)

    # Initial frame data
    initial_data = [
        create_scatter3d(xyz_all_frames[0], idx_parts, color)
        for idx_parts, color in zip(idx_parts_list[0], color_list)
    ]
    bounding_box_trace = go.Scatter3d(
    x=[min_xyz[0], max_xyz[0]],
    y=[min_xyz[1], max_xyz[1]],
    z=[min_xyz[2], max_xyz[2]],
    mode='markers',
    marker=dict(size=0.1, color='rgba(0,0,0,0)'),
    showlegend=False
    )
    initial_data.append(bounding_box_trace)
    # Create frames for animation
    frames_data = [
        create_frame(xyz_frame, idx_parts_list[i], color_list, str(i))
        for i, xyz_frame in enumerate(xyz_all_frames)
    ]

    # Build full figure
    fig = go.Figure(
        data=initial_data,
        layout=go.Layout(
            scene=dict(
                xaxis_title="X",
                yaxis_title="Y",
                zaxis_title="Z",
            ),
            updatemenus=create_play_pause_buttons(),
            sliders=create_slider(frames),
        ),
        frames=frames_data,
    )

    fig.show()
    fig.write_html(output_path)
    print(f"Saved animation to: {output_path}")


create_3d_animation(xyz_rotated, idx_parts, color_list)



Saved animation to: D:/Documents/gaussian_model_output//fly_to_bee/animated_plot_bee.html


In [ ]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)



from Camera import Camera
from Frame import Frame
import Plotters
import Utils as Utils

from render.GaussianSplat import GaussianSplat


import numpy as np
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import Plotters
import plotly
import plotly.graph_objects as go
import scipy.io
import pandas as pd
from Render import Render
from plyfile import PlyData

import pickle
import numpy as np
import math

import matplotlib.pyplot as plt
import Utils
%matplotlib qt


path = 'C:/Users/Roni/Documents/gs_input/frames_model.pkl'
dict_path  = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov30_2024_11_12_darkan/frames_model.pkl'

# dict_path = 'D:/Documents/data_for_gs/fly_gray/dict/frames_model.pkl'
image_path = 'D:/Documents/data_for_gs/fly_gray/'
image_path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov30_2024_11_12_darkan/'
file_name = 'model_rotation_lr_center0.07_densify_grad_threshold_0.00035_long_wing_thin_y2_weight' # model and deform, xyz together
file_name = 'model_rotation_lr_center0.07_densify_grad_threshold_0.00035_re' # model and deform, xyz together

file_name = "densify_until_2000_densify_from_500_iterations_20000_default_opacity_intvl_50_opacity_lr0.1"
input_file = f"D:/Documents/gaussian_model_output/opacity_3dgs/1447/{file_name}/point_cloud/iteration_10/point_cloud.ply"

path_angles = f'D:/Documents/gaussian_model_output/mdoel_le_deform/{file_name}_angles.pkl'

with open(dict_path,'rb') as f:
    frames = pickle.load(f)

vertices_list = []
image_list = []

gaussian_list = []
for frame in range(1447,1448):
    input_file = f"D:/Documents/gaussian_model_output/opacity_3dgs_noaa/1447/{file_name}/point_cloud/iteration_5000/point_cloud.ply"
    vertices = PlyData.read(input_file)["vertex"]
    vertices_list.append(np.column_stack((vertices['x'],vertices['y'],vertices['z'])))
    frames_per_cam = [Frame(image_path,frame,cam, frames_dict = frames)  for cam in range(4)]
    image_list.append(frames_per_cam)


frame = 1447
frame0 = 1447
# input_file = f'D:/Documents/gaussian_model_output/mdoel_le_deform/{frame}/{file_name}/point_cloud/iteration_900/point_cloud.ply'

frame_data = image_list[frame - frame0] 

ew_to_lab = frames[frame][1][list(frames[frame][1].keys())[0]]['ew_to_lab']
xyz = vertices_list[frame - frame0]
xyz_rotated = (ew_to_lab @ xyz.T).T


gs = GaussianSplat(input_file,image_size = frame_data[0].image_size)
rgb = gs.color
opacity = gs.opacity
grayscale = rgb[:,0]
grayscale = (grayscale - grayscale.min()) / (grayscale.max() - grayscale.min())
grayscale = grayscale[grayscale <1]
fig = go.Figure()

Plotters.scatter3d(fig,xyz_rotated,grayscale,3,'body',show_colorbar = False)
fig.show()
fig.data = []
fig.layout = {}
ax = Plotters.plot_projections(xyz,frame_data,color = 'lime')



# frames_per_cam = [Frame(image_path,frame,cam_num, frames_dict = frames) for cam_num in range(4)]


In [56]:
frames_data = []
frames_color = []

for frame in range(1430,1600): 

    
    with open(dict_path,'rb') as f:
        frames = pickle.load(f)
    
    ew_to_lab = frames[frame][1][list(frames[frame][1].keys())[0]]['ew_to_lab']

 
    input_file = f'{path_output}/{model_name}/{frame}/{file_name}/point_cloud/iteration_{iteration}/point_cloud.ply'

    image_size = frames_per_cam[0].image_size
    gs = GaussianSplat(input_file,image_size = image_size)  

    vertices = PlyData.read(input_file)["vertex"]
    vertices = gs.xyz

    xyz = (ew_to_lab @ vertices.T).T


    rgb = gs.color
    opacity = gs.opacity
    grayscale = rgb[:,0]
    grayscale = (grayscale - grayscale.min()) / (grayscale.max() - grayscale.min())

    frames_data.append({'x':xyz[:,0],'y':xyz[:,1],'z':xyz[:,2]})
    frames_color.append(grayscale)

In [ ]:
# file_name = 'gradth_0.0002_dense_until_15000_densify_from_500' 2dgs
file_name = 'anti_al_default_opacity_intvl_50_opacity_lr0.1'
file_name = 'default_opacity_intvl_50_opacity_lr0.1'
file_name = 'anti_al_iterations_5000  default_opacity_intvl_50_opacity_lr0.1'
file_name = 'densify_until_2000_densify_from_500_iterations_20000_default_opacity_intvl_50_opacity_lr0.1'
file_name = 'gray_dens_until4000_opacity_lr0.1_dens_interval100_opacity_interval20'

file_name = 'position_lr_init_1.6e-05ldist5' #2dgs
file_name = 'gray_dens_until5000_opacity_lr0.05_dens_interval100_opacity_interval20_lambda_normal0lambda_dist10'
file_name = 'densify_until_2000_densify_from_2000_iterations_2000_default_opacity_intvl_15_opacity_lr0.1'
input_file = f"D:/Documents/gaussian_model_output/opacity_3dgs/1400/{file_name}/point_cloud/iteration_10000/point_cloud.ply"
input_file = f"D:/Documents/gaussian_model_output/opacity_3dgs/1447/{file_name}/point_cloud/iteration_10/point_cloud.ply"
input_file = f"D:/Documents/gaussian_model_output/opacity_3dgs_noaa_sweep/1400/{file_name}/point_cloud/iteration_500/point_cloud.ply"
# input_file = f"I:/My Drive/Rese
# arch/gaussian_splatting/gaussian_splatting_output/hull_output/opacity_3dgs_noaa/1447/{file_name}/point_cloud/iteration_2000/point_cloud.ply"
input_file = f"G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/hull_output/3dgs_bw/1400/{file_name}/point_cloud/iteration_1000/point_cloud.ply"
# input_file = f"G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/hull_output/2dgs_normals/1400/{file_name}/point_cloud/iteration_10/point_cloud.ply"
# input_file = f"G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/fly_gaussian/2d_output/{file_name}/1407/point_cloud/iteration_10000/point_cloud.ply"
# I:\My Drive\Research\gaussian_splatting\gaussian_splatting_output\fly_gaussian\2d_output\iterations_5000_densification_interval_1000_ld_5_densify_until4000_ln05_lr001_ori_100_dr1\1407\point_cloud\iteration_5000


input_file = f'D:/Documents/gaussian_model_output/only_model3/{frame}/{file_name}/point_cloud/iteration_500/point_cloud.ply'


file_name= 'scale_model0.05_model_rotation_lr_0.1_23_1_3_1' # only model
input_file = f'D:/Documents/gaussian_model_output/only_model3/{frame}/{file_name}/point_cloud/iteration_500/point_cloud.ply'


file_name = 'scale_model0.05_model_rotation_lr_0.1_23_1_3_1_deform_xyz' # model and deform, xyz together
input_file = f'D:/Documents/gaussian_model_output/only_model4/{frame}/{file_name}/point_cloud/iteration_500/point_cloud.ply'


file_name = 'scale_model0.05_model_rotation_lr_0.1_23_1_3_1_deform_xyz_v2' # model and deform, xyz together
input_file = f'D:/Documents/gaussian_model_output/only_model4/{frame}/{file_name}/point_cloud/iteration_700/point_cloud.ply'



file_name = 'scale_model0.05_model_rotation_lr_0.1_scale0823_1_3_1_deform_xyz_v2' # model and deform, xyz together
input_file = f'D:/Documents/gaussian_model_output/only_model5/{frame}/{file_name}/point_cloud/iteration_700/point_cloud.ply'

file_name = 'model_rotation_lr_center0.07' # model and deform, xyz together
input_file = f'D:/Documents/gaussian_model_output/model_with_deform/{frame}/{file_name}/point_cloud/iteration_1200/point_cloud.ply'


file_name = 'init_xyz_late0.00016_densify_grad_threshold_0.00025_long_wing' # model and deform, xyz together
input_file = f'D:/Documents/gaussian_model_output/model_with_deform_twist_clamp5/{frame}/{file_name}/point_cloud/iteration_400/point_cloud.ply'


file_name = 'init_xyz_late0.00016_densify_grad_threshold_0.00025_long_wing' # model and deform, xyz together
input_file = f'D:/Documents/gaussian_model_output/mdoel_le_deform/{frame}/{file_name}/point_cloud/iteration_900/point_cloud.ply'

file_name = 'model_rotation_lr_center0.07_densify_grad_threshold_0.00035_long_wing_thin' # model and deform, xyz together
input_file = f'D:/Documents/gaussian_model_output/mdoel_le_deform/{frame}/{file_name}/point_cloud/iteration_900/point_cloud.ply'


file_name = 'model_rotation_lr_center0.07_densify_grad_threshold_0.00035_long_wing_thin_yless' # model and deform, xyz together
input_file = f'D:/Documents/gaussian_model_output/mdoel_le_deform/{frame}/{file_name}/point_cloud/iteration_1400/point_cloud.ply'


file_name = 'model_rotation_lr_center0.07_densify_grad_threshold_0.00035_long_wing_thin_y2' # model and deform, xyz together
input_file = f'D:/Documents/gaussian_model_output/mdoel_le_deform/{frame}/{file_name}/point_cloud/iteration_900/point_cloud.ply'


# file_name = 'scale_model0.05_model_rotation_lr_0.1_23_1_3_1_xyz_after_model' # model and deform, xyz together
# input_file = f'D:/Documents/gaussian_model_output/only_model4/{frame}/{file_name}/point_cloud/iteration_2500/point_cloud.ply'


# file_name = 'random_initilize_featrot'
# input_file = f'D:/Documents/gaussian_model_output/only_model/{frame}/{file_name}/point_cloud/iteration_1000/point_cloud.ply'







In [349]:
import open3d as o3d
path_to_mesh = 'D:/Documents/model_wings_center_1430_1900/model/mesh/wing2.stl'
mesh = o3d.io.read_triangle_mesh(path_to_mesh)
pt_cloud = np.asarray(mesh.vertices)
fig = go.Figure()
Plotters.scatter3d(fig,pt_cloud,grayscale,3,'vol',show_colorbar = False)
fig.show()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D WARNING] Unable to load file D:/Documents/model_wings_center_1430_1900/model/mesh/wing2.stl with ASSIMP


In [228]:
# file_name = 'gradth_0.0002_dense_until_15000_densify_from_500' 2dgs
# file_name = 'gray_dens_until4000_opacity_lr0.05_dens_interval100_opacity_interval20'
# file_name = 'gradth_0.0002_dense_until_15000_densify_from_1000' #2dgs

# input_file = f"G:/My Drive/Research/gaussian_splatting/gaussian_splatting_output/hull_output/2dgs_sweep_2/1400/{file_name}/point_cloud/iteration_5000/point_cloud.ply"
image_size = frames_per_cam[0].image_size
gs = GaussianSplat(input_file,image_size = frames_per_cam[0].image_size)  
cam = frames_per_cam[0]
# gs.save_gs(name = '_2dsplat')
render = Render(gs,cam,tiles = [1,10],block_xy = [16,16], image_size = [160,160],gaus3d = False,filtersze = 0.707)

image,depth = render.render_image()
plt.figure(),plt.imshow(image)
plt.figure(),plt.imshow(depth/np.max(depth))
# file_name = 'gradth_0.0002_dense_until_15000_densify_from_500' 2dgs


(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x1ab186d5cf0>)

In [684]:
images = [frames_per_cam[idx].project_with_proj_mat(xyz) for idx in range(4)]
images

[array([[ 70.84391571,  81.89843613, 159.49999973, 159.5       ],
        [ 74.45092477,  81.46709241, 159.49999973, 159.5       ],
        [ 74.12799459,  82.90606251, 159.49999973, 159.5       ],
        ...,
        [ 65.57592582,  90.99322091, 159.49999973, 159.5       ],
        [ 68.41051938,  90.56989505, 159.49999973, 159.5       ],
        [ 68.11962237,  90.33670856, 159.49999973, 159.5       ]]),
 array([[101.35906097,  99.62034764, 159.49999954, 159.5       ],
        [ 96.50811829, 100.95263516, 159.49999954, 159.5       ],
        [ 97.01317437,  99.78800469, 159.49999954, 159.5       ],
        ...,
        [107.29264344,  57.63287138, 159.49999954, 159.5       ],
        [103.40065493,  57.39059952, 159.49999954, 159.5       ],
        [103.78625906,  57.64036899, 159.49999954, 159.5       ]]),
 array([[ 72.64397116, 107.51569514, 159.49999958, 159.5       ],
        [ 74.60664012, 106.35919869, 159.49999958, 159.5       ],
        [ 75.8639238 , 107.00448893, 159.49999

In [ ]:
frames_data = []
frames_color = []

for frame in range(1430,1710): 

    
    with open(dict_path,'rb') as f:
        frames = pickle.load(f)
    
    ew_to_lab = frames[frame][1][list(frames[frame][1].keys())[0]]['ew_to_lab']

 
    # file_name = 'model_rotation_lr_center0.07_densify_grad_threshold_0.00035_long_wing_thin_y2' # model and deform, xyz together
    input_file = f'{path_output}/{model_name}/{frame}/{file_name}/point_cloud/iteration_{iteration}/point_cloud.ply'


    image_size = frames_per_cam[0].image_size

    gs = GaussianSplat(input_file,image_size = image_size)  
    vertices = PlyData.read(input_file)["vertex"]
    vertices = gs.xyz

    xyz = (ew_to_lab @ vertices.T).T


    rgb = gs.color
    opacity = gs.opacity
    grayscale = rgb[:,0]
    grayscale = (grayscale - grayscale.min()) / (grayscale.max() - grayscale.min())

    frames_data.append({'x':xyz[:,0],'y':xyz[:,1],'z':xyz[:,2]})
    frames_color.append(grayscale.astype(float))

In [255]:
grayscale.astype(float)

array([0.00412412, 0.04984919, 0.00481547, ..., 0.6693898 , 0.77232028,
       0.75506949])

In [57]:
import plotly.graph_objects as go
import numpy as np

frames = range(1430,1600)
# Number of frames
# Generate random data for each frame within a fixed range

# Compute global min/max across all frames to ensure equal scaling
x_min, x_max = min(min(f["x"]) for f in frames_data), max(max(f["x"]) for f in frames_data)
y_min, y_max = min(min(f["y"]) for f in frames_data), max(max(f["y"]) for f in frames_data)
z_min, z_max = min(min(f["z"]) for f in frames_data), max(max(f["z"]) for f in frames_data)

# Ensure equal scaling by using the same range for all axes
axis_min = min(x_min, y_min, z_min)
axis_max = max(x_max, y_max, z_max)

# Create the figure
fig = go.Figure(


    data=[
        go.Scatter3d(
            x=frames_data[0]["x"],
            y=frames_data[0]["y"],
            z=frames_data[0]["z"],
            mode="markers",
            marker=dict(size=3, opacity=0.8, color = frames_color[0],colorscale = 'gray'),

        )
    ],
    layout=go.Layout(
        #         scene=dict(
        #     xaxis=dict(title="X", range=[x_min, x_max]),
        #     yaxis=dict(title="Y", range=[y_min, y_max]),
        #     zaxis=dict(title="Z", range=[z_min, z_max]),
        #     aspectmode="data",
        # ),
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}],
                        "label": "Play",
                        "method": "animate",
                    },
                    {
                        "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                        "label": "Pause",
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top",
            }
        ],
        sliders=[
            {
                "active": 0,
                "steps": [
                    {
                        "args": [[str(i)], {"frame": {"duration": 100, "redraw": True}, "mode": "immediate"}],
                        "label": str(frame),
                        "method": "animate",
                    }
                    for i,frame in enumerate(frames)
                ],
            }
        ],
    ),
    frames=[
        go.Frame(
            data=[
                go.Scatter3d(
                    x=frame["x"],
                    y=frame["y"],
                    z=frame["z"],
                    mode="markers",
                    marker=dict(size=3, opacity=1, color = frames_color[i],colorscale = 'gray' ),
                )
            ],
            name=str(i),
        )
        for i, frame in enumerate(frames_data)
    ],
)

# Show the plot
fig.show()
path_to_save = 'D:/Documents/model_gaussian_splatting/plots/animated_plot700.html'
fig.write_html("animated_plot.html")



In [224]:
frame0 = 1430
# Compute global min/max across all frames to ensure equal scaling
x_min, x_max = min(min(f["x"]) for f in frames_data), max(max(f["x"]) for f in frames_data)
y_min, y_max = min(min(f["y"]) for f in frames_data), max(max(f["y"]) for f in frames_data)
z_min, z_max = min(min(f["z"]) for f in frames_data), max(max(f["z"]) for f in frames_data)

fig = go.Figure()

# Add traces, one for each slider step
for i, frame in enumerate(frames_data):
    # Main data points trace
    fig.add_trace(
        go.Scatter3d(
            visible=False,
            x=np.hstack((frame["x"])),
            y=np.hstack((frame["y"])),
            z=np.hstack((frame["z"])),
            mode="markers",
            marker=dict(size=3, opacity=1, color=frames_color[i], colorscale="gray"),
        )
    )

    # Bounding box trace (invisible but included for scaling)
    fig.add_trace(
        go.Scatter3d(
            visible=False,
            x=[x_min, x_max],
            y=[y_min, y_max],
            z=[z_min, z_max],
            mode="markers",
            opacity=0,  # Keep it invisible
        )
    )

# Ensure the first frame is visible
fig.data[0].visible = True
fig.data[1].visible = True

# Create and add slider
steps = []
for i in range(len(frames_data)):  # Iterate over frames
    visibility = [False] * len(fig.data)  # Default: Hide everything
    visibility[i * 2] = True  # Show main data trace
    visibility[i * 2 + 1] = True  # Show bounding box trace

    step = dict(
        method="update",
        args=[
            {"visible": visibility},  # Update visibility
            {"title": f"Frame: {i + frame0}"}  # Update title
        ],
        label=str(i + frame0)  # Correct label format
    )
    steps.append(step)

# Add the slider to the layout
sliders = [dict(
    active=0,  # Start at first frame
    currentvalue={"prefix": "Frame: "},
    pad={"t": 1},
    steps=steps,
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

path_to_save = 'D:/Documents/model_gaussian_splatting/plots/animated_plot_2000_deformed.html'
fig.write_html(path_to_save)



FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Documents\\model_gaussian_splatting\\plots\\animated_plot_2000_deformed.html'

In [ ]:
import plotly.graph_objects as go
import numpy as np

frames = range(1430,1531)
# Number of frames
# Generate random data for each frame within a fixed range

# Compute global min/max across all frames to ensure equal scaling
x_min, x_max = min(min(f["x"]) for f in frames_data), max(max(f["x"]) for f in frames_data)
y_min, y_max = min(min(f["y"]) for f in frames_data), max(max(f["y"]) for f in frames_data)
z_min, z_max = min(min(f["z"]) for f in frames_data), max(max(f["z"]) for f in frames_data)

# Ensure equal scaling by using the same range for all axes
axis_min = min(x_min, y_min, z_min)
axis_max = max(x_max, y_max, z_max)

# Create the figure
fig = go.Figure(


    data=[
        go.Scatter3d(
            x=frames_data[0]["x"],
            y=frames_data[0]["y"],
            z=frames_data[0]["z"],
            mode="markers",
            marker=dict(size=3, opacity=0.8, color = frames_color[0],colorscale = 'gray'),

        )
    ],
    layout=go.Layout(
        #         scene=dict(
        #     xaxis=dict(title="X", range=[x_min, x_max]),
        #     yaxis=dict(title="Y", range=[y_min, y_max]),
        #     zaxis=dict(title="Z", range=[z_min, z_max]),
        #     aspectmode="data",
        # ),
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}],
                        "label": "Play",
                        "method": "animate",
                    },
                    {
                        "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                        "label": "Pause",
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top",
            }
        ],
        sliders=[
            {
                "active": 0,
                "steps": [
                    {
                        "args": [[str(i)], {"frame": {"duration": 100, "redraw": True}, "mode": "immediate"}],
                        "label": str(frame),
                        "method": "animate",
                    }
                    for i,frame in enumerate(frames)
                ],
            }
        ],
    ),
    frames=[
        go.Frame(
            data=[
                go.Scatter3d(
                    x=frame["x"],
                    y=frame["y"],
                    z=frame["z"],
                    mode="markers",
                    marker=dict(size=3, opacity=1, color = frames_color[i],colorscale = 'gray' ),
                )
            ],
            name=str(i),
        )
        for i, frame in enumerate(frames_data)
    ],
)

# Show the plot
fig.show()
path_to_save = 'D:/Documents/model_gaussian_splatting/plots/animated_plot700.html'
fig.write_html("animated_plot.html")



In [ ]:
# fig.write_html("animated_plot.html")
b[2]

Scatter3d({
    'hoverinfo': 'none',
    'name': '',
    'opacity': 0,
    'showlegend': False,
    'visible': True,
    'x': [0, 1],
    'y': [0, 1],
    'z': [0, 1]
})

In [163]:
path_angles = 'D:/Documents/gaussian_model_output/mdoel_le_deform/model_rotation_lr_center0.07_densify_grad_threshold_0.00035_long_wing_thin_y2_angles.pkl'

with open(path_angles, 'rb') as handle:
    b = pickle.load(handle)

plt.figure(),plt.plot(b[3][:,0:5])
# plt.plot(b[0][:,0])
b[0].shape

print(b[0][-41],b[1][-41],b[2][-41])

[ -8.134133  -38.76222    -2.9296703] [ -5.5953383 -47.830235  -15.683396 ] [-101.813    -36.16147   14.065  ]


In [173]:
b[2]

array([[-104.17224  ,  -29.155941 ,   14.635098 ],
       [-104.27701  ,  -30.917753 ,   14.409257 ],
       [-104.301315 ,  -31.257793 ,   14.5323715],
       [-105.145805 ,  -32.87673  ,   15.19736  ],
       [-106.79624  ,  -35.29106  ,   15.383101 ],
       [-107.26133  ,  -36.36495  ,   15.863743 ],
       [-107.56719  ,  -36.988667 ,   15.776338 ],
       [-106.545105 ,  -37.417984 ,   15.213478 ],
       [-106.80101  ,  -36.950546 ,   15.253748 ],
       [-106.02583  ,  -35.14723  ,   14.45921  ],
       [-104.801704 ,  -32.332996 ,   14.33518  ],
       [-103.635414 ,  -30.063232 ,   14.175861 ],
       [-102.87352  ,  -30.888815 ,   14.152831 ],
       [-103.08027  ,  -30.870651 ,   13.7398815],
       [-103.08053  ,  -30.52155  ,   13.798071 ],
       [-103.29023  ,  -30.488523 ,   13.8770895],
       [-103.31814  ,  -30.743002 ,   13.82886  ],
       [-104.1395   ,  -31.494492 ,   14.324403 ],
       [-104.22106  ,  -31.503124 ,   14.329618 ],
       [-104.763695 ,  -32.9213

## Elliptical Gaussian (from EWA article) - Math explained


we define an elliptical gaussian: 

Equation 1: $G_\mathbf{V}(\mathbf{x} - \mathbf{p}) = \frac{1}{2 \pi |\mathbf{V}|} \frac{1}{2} e^{-\frac{1}{2} (\mathbf{x} - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{x} - \mathbf{p})}.$

* $V$ is the covariance matrix
* $\mathbf{x}$ - a 3d coordinate (in gaussian axes)
* $\mathbf{p}$ - the mean of the gaussian 
### Affine projection of the gauusian
We define an arbitrary affine mapping from object space $ \mathbf{u} = \Phi(\mathbf{x}) $, where $ \Phi(\mathbf{x}) = \mathbf{Mx + c} $, 

and we substitute $ \mathbf{x} = \Phi^{-1}(\mathbf{u}) $. Since $ \Phi^{-1}(\mathbf{u}) = \mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) $, we get:

$$
G_\mathbf{V}(\Phi^{-1}(\mathbf{u}) - \mathbf{p}) = 
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \mathbf{c} - \mathbf{Mp})^T \mathbf{M}^{-1} \mathbf{V}^{-1} (\mathbf{M}^{-1})^T (\mathbf{u} - \mathbf{c} - \mathbf{Mp})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \Phi(\mathbf{p}))^T (\mathbf{MVM}^T)^{-1} (\mathbf{u} - \Phi(\mathbf{p}))}.
$$

from this, to ease the writing we define equation 2: 

Equation 2: $G \mathbf{V}(\Phi^{-1}(u) - \mathbf{p}) = \frac{1}{|\mathbf{M}^{-1}|} G_{ \mathbf{M} \mathbf{V} \mathbf{M}^T} (u - \Phi(\mathbf{p})).$

#### The covariance matrix
$V$, the covariance matrix is a symetrix matrix that can be ragarded as a transformation. it scales and shears the data (thus defining the orientation and ratio between the gaussian principal axes). 
As mentioned above, the gaussian is transformed using an affine mapping $\Phi(X)$. Usually we use this kind of mapping to transform a vector or a 3D point to a new FoR (change basis). Because the covariance matrix is a transformation, in order to change its FoR (basis), we will perform $\mathbf{V_{1}} = MVM^{T}$. We first multiply $VM^{T}$  - transform (whatever xyz we multiply) from camera to body ($M^{T}$) and then multiply by $V$ which is in object coordinates - now we have coordinates in object basis after shear and scale. we then multiply by $M$ again to transform back to camera coordinates. 


### Projection


![alt text](ray_ewa.png)
#### Ray space

We converted to camera space, now we want to convert to ray space. The coordinates in ray space are defined such that $x_0, x_1$ are the coordinates on the image plane and the devision by $u_2$ adds prespective. $u_3$ is then defined as the euclidean distance from the point to the origin (camera). 


$$ \text{ray space to camera: }
\begin{pmatrix}
x_0 \\
x_1 \\
x_2 \\

\end{pmatrix} = m(u) = \begin{pmatrix}
u_0 / u_2 \\
u_1/u_2 \\
||(u_0, u_1, u_2)||^{T} \\
\end{pmatrix}

\text{,camera to ray space: }
\begin{pmatrix}
u_0 \\
u_1 \\
u_2 \\

\end{pmatrix} = m^{-1}(x) = \begin{pmatrix}
x_0/||(x_0,x_1,1)^T|| \cdot x_2 \\
x_1/||(x_0,x_1,1)^T|| \cdot x_2 \\
1/||(x_0,x_1,1)^T|| \cdot x_2 \\

\end{pmatrix}


$$
  
Ray space is defined with 3 coordinates $(x_0,x_1,x_2)$ where $x_0$ and $x_1$ represents the location of the pixel and $x_3$ is the euclidean distance from the camera origin to $(x_1,x_2). This transformation is not affine due to the devision by $u_2$, parallel lines are not preserved and the shape of the gaussian will be distorted. (in addition, the dimention is problematic).
In order to solve this a local affine approximation $\mathbf{m_{u_k}}$ of the projection transformation is defined. The approximation is the first to terms of the Tayor expansion of $\mathbf{m}$ at point $\mathbf{u}_{k}$:
$$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
where $\mathbf x_k=\mathbf m(\mathbf u_k)$ is the mean of the gaussian in ray space (can be transformed using the mapping m, the projection matrix) and the Jacobian $ J_{u_k} = \frac{\partial m}{\partial u}(u_k) $

### The gaussian mapping
#### Object to camera coordinates (as detailed in Affine projection of the gauusian)

The reconstruction kernels are initially given in object space, which has coordinates $t =(t_0,t_1,t_2)^T$ . 
* Gaussian reconstruction kernels in object space: $r"_k (t)=G_{V"_k} (t − t_k)$, where $t_k$ are the voxel positions in object space
* Object coordinates are transformed to camera coordinates using an affine mapping $\mathbf u = \phi(t)$, called viewing transformation. It is defined by a matrix $\mathbf W$ and a translation vector $\mathbf d$ as $\phi(t) = Wt + d$. We transform the reconstruction kernels $G_{V"k} (t − t_k)$ to camera space by substituting $t = \phi ^{−1}(u)$ and using Equation 2 : 

$G \mathbf{V"_{k}}(\Phi^{-1}(u) - t_k) = \frac{1}{|\mathbf{W}^{-1}|} G_{V'_k } (\mathbf u - \mathbf u_k) = r'_k(u\mathbf).$

where $u_k = \phi(\mathbf t_k)$ is the center of the Gaussian in camera coordinates and $r′_k(\mathbf u)$ denotes the reconstruction kernel in camera space. According to equation 2, the covariance matrix in camera coordinates $V′_k$ is given by $V′_k = WV"_kW^T$ (exactly as detailed in Affine projection of the gauusian).

#### Camera to ray space
Reminder: the local affine transformation - $$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
$\mathbf{m_{u_k}\mathbf(u)}$ transforms from camera to ray, hence $x_k=m(u_k)$ where $x_k$ is the center of the gaussian in ray space. 
since $\mathbf x = m(\mathbf u)$ we get that $ u = \mathbf m^{-1}(\mathbf(x))$ hence: 
$$ u = \mathbf m^{-1}(x) = \mathbf(x - x_k)\cdot \mathbf J^{-1}_{\mathbf u_k} + \mathbf u_k $$
$$ r_k(\mathbf x) = \frac{1}{|\mathbf W^{-1}|}\cdot G_{\mathbf V'_k}(\mathbf{(x - x_k)\cdot J^{-1}_{\mathbf u_k} + \mathbf u_k - \mathbf u_k}) = \\
\frac{1}{|\mathbf W^{-1}||\mathbf J^{-1}|}G_{\mathbf V'_k}(J^{-1}_{\mathbf u_k}\mathbf{(x - x_k)})\\$$

and the new covariance matrix is defined:  $Vk = JV'_{k}$ $J^T = JWV"_kW^TJ^T$.
where V"_k is the original covariance matrix





### Perspective Projection and Jacobian Derivation

In a pinhole camera model, the perspective projection of a 3D point \((X, Y, Z)\) into 2D image coordinates \((x, y)\) can be expressed as:

$$
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix} = \frac{1}{Z} \cdot
\begin{bmatrix}
f_x \cdot X + c_x \cdot Z \\
f_y \cdot Y + c_y \cdot Z \\
Z
\end{bmatrix}
$$

Where:

* $(x, y)$  : are the projected 2D coordinates on the image plane.
* $(X, Y, Z)$  : are the coordinates in the 3D space. 
* $f_x , f_y$  : are the focal lengths in the x and y directions, respectively. 
* $c_x , c_y$  : are the coordinates of the principal point (optical center) in the image. 

### Derivation of the Jacobian

To derive the Jacobian, we compute the partial derivatives of the projected 2D coordinates \((x, y)\) with respect to the 3D coordinates \((X, Y, Z)\).

1. **From the perspective projection equations**:
   - For \(x\):
   $$
   x = \frac{f_x \cdot X + c_x \cdot Z}{Z}
   $$
   - For \(y\):
   $$
   y = \frac{f_y \cdot Y + c_y \cdot Z}{Z}
   $$

2. **Calculating the derivatives**:
   - The derivatives with respect to \(Z\) yield:
     - For \(x\):
     $$
     \frac{\partial x}{\partial X} = \frac{f_x}{Z}, \quad \frac{\partial x}{\partial Y} = 0, \quad \frac{\partial x}{\partial Z} = -\frac{f_x \cdot X}{Z^2}
     $$
     - For \(y\):
     $$
     \frac{\partial y}{\partial X} = 0, \quad \frac{\partial y}{\partial Y} = \frac{f_y}{Z}, \quad \frac{\partial y}{\partial Z} = -\frac{f_y \cdot Y}{Z^2}
     $$

3. **Forming the Jacobian**:
   Using the derivatives calculated above, the Jacobian matrix \(J\) becomes:
   $$
   J = \begin{bmatrix}
   \frac{f_x}{Z} & 0 & -\frac{f_x \cdot X}{Z^2} \\
   0 & \frac{f_y}{Z} & -\frac{f_y \cdot Y}{Z^2}
   \end{bmatrix}
   $$


### 2D Projection 
We calculate the conics coeffitient for each gaussian : $$Ax^2+Bxy+Cy^2 = 0$$, where $A = \frac{\sigma_y^2}{|V|}$, $B = \frac{\sigma_xy}{|V|}$ and  $C = \frac{\sigma_x^2}{|V|}$
we then multiply by the distace between the gaussiam and the pixel. this is the same as: 
$-\frac{1}{2} (\mathbf{x} - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{x} - \mathbf{p})$ - the exponent of the gaussian

wIncorporating the scaled conic, which represents the 2D Gaussian as an ellipse in the Gaussian equation, we can express the effect of every Gaussian on the pixel's color as follows:

$$
G_{\mathbf{V}}(\mathbf{x}) = \frac{1}{(2\pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} ( Ax^2+Bxy+Cy^2)}
$$

This equation captures how each Gaussian contributes to the pixel's color based on its distance from the pixel and the shape defined by the covariance matrix.

## tiles
To render the image we devide it to tiles of $ 16 X 16 $, for each tile we sort the gaussians according to the distance from the camera. we then sample all gaussians effect per pixel in the tile (using the power equation to calculate the opacity). We sum the gaussians color values and multiply by the opacity (doing alpha blending, each time we update the opacity according to the already calculated gaussians $color = RGB * \alpha *\alpha_{blend})$ were for eacg gaussian we add we update $\alpha_{blend} =  \alpha_{blend}*(1-\alpha) $ (when we reach 0, its opaque). to later do the alpha blend we pick for each tile only the gaussians that intersect with the tile. since gaussians are not bounded, we bound them with 3 $\sigma_x$ and 3 $\sigma_y$ bounding box in each axis. 

